In [1]:
# Import Pandas
import pandas as pd

#Set Relative Path
import sys
sys.path.append("../src")

In [2]:
# Import data
df_dataK = pd.read_csv(r'../data/d03_cleaned_data/datall.csv')
df_dataK.head()

,text,emotion
0,i got up feeling horny this morning,love
1,i will go to my mailbox and talk to the mailma...,happy
2,i understand that any of my extremely positive...,happy
3,i suggest you give it a listen i feel like i a...,love
4,i have been with petronas for years i feel tha...,happy


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.preprocessing import LabelEncoder
from wordcloud import STOPWORDS
STOPWORDS.update(['feel','feeling','im',",","t","u","2","'","&amp;","-","...","s"])

# Set X and y
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words=STOPWORDS)
tfidf_matrix = tf.fit_transform(df_dataK['text'])
X = tfidf_matrix

Labelenc = LabelEncoder()
df_dataK['emotion'] = Labelenc.fit_transform(df_dataK['emotion'])
y = df_dataK['emotion']
print (y.unique())

/home/apprenant/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['amp', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'let', 'll', 'mustn', 're', 'shan', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


[3 2 4 0 1 5]


In [4]:
# Prediction
#userinput= str(input()) 
userinput = "i love you so much!!!!"
print (userinput)
x = tf.transform([userinput])

i love you so much!!!!


In [5]:
from reg_lin import get_metrix
from sklearn.linear_model import LogisticRegression ,SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from keras.models import Sequential 
from tensorflow.keras import layers, preprocessing
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score,accuracy_score
models = [  LogisticRegression(),
            SGDClassifier(),
            RandomForestClassifier(),
            XGBClassifier(),
            Sequential()]
place = 4
savedmodel=[]
count = 0
for modelset in models :
    if count == place:
        
        df_dataKKeras = df_dataK . sample (15000)
        y = df_dataKKeras['emotion']
        df_dataKKeras = df_dataKKeras[['text']]
        tk = preprocessing.text.Tokenizer()
        tk.fit_on_texts(df_dataKKeras['text'])
        X = tk.texts_to_matrix(df_dataKKeras['text'], mode='tfidf')
        x = tk.texts_to_matrix([userinput], mode='tfidf')
    
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=None)
        model = modelset
        model.add(layers.Dense(160, activation = 'relu', input_shape= [X_train.shape[1]]))
        model.add(layers.Dense(80, activation = 'relu'))
        model.add(layers.Dense(40, activation='relu'))
        model.add(layers.Dense(6,activation='softmax'))
        model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy']) 
        model.fit(X_train,y_train,epochs=10, batch_size=100)

        y_train_pred = model.predict_classes(X_train)
        y_test_pred = model.predict_classes(X_test)
        
        print (model)

        print (" Train accuracy = {}" 
        .format(round(accuracy_score(y_train, y_train_pred),3)))

        print (" Test accuracy = {}" 
        .format(round(accuracy_score(y_test, y_test_pred),3)))
    if count != place:
        model = get_metrix(y,X,modelset)
    pred = model.predict(x)
    print (pred)
    if count == place:
        pred = Labelenc.inverse_transform([round(pred[0][0])])
    if count != place:
        pred = Labelenc.inverse_transform(pred)
    print (pred)
    savedmodel.append(model)
    count += 1
    print("\n")

2021-05-21 11:12:50.533 WARNING root: 
  command:

    streamlit run /home/apprenant/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py [ARGUMENTS]


InternalHashError: module '__main__' has no attribute '__file__'

While caching the body of `get_metrix()`, Streamlit encountered an
object of type `builtins.function`, which it does not know how to hash.

**In this specific case, it's very likely you found a Streamlit bug so please
[file a bug report here.]
(https://github.com/streamlit/streamlit/issues/new/choose)**

In the meantime, you can try bypassing this error by registering a custom
hash function via the `hash_funcs` keyword in @st.cache(). For example:

```
@st.cache(hash_funcs={builtins.function: my_hash_func})
def my_func(...):
    ...
```

If you don't know where the object of type `builtins.function` is coming
from, try looking at the hash chain below for an object that you do recognize,
then pass that to `hash_funcs` instead:

```
Object of type builtins.function: <function get_metrix at 0x7fcac31e4f70>
```

Please see the `hash_funcs` [documentation]
(https://docs.streamlit.io/en/stable/caching.html#the-hash-funcs-param

In [25]:
del x,X,y,X_test,X_train,y_test,y_train,y_test_pred,y_train_pred,df_dataK,df_dataKKeras

In [30]:
# Import data
df_dataK = pd.read_csv(r'../data/d03_cleaned_data/datall_test.csv')

X_test = tf.transform(df_dataK['text'])
y_test = Labelenc.transform(df_dataK['emotion'])

count = 0
place = 4 

for model in savedmodel:

    print (model)
    if count != place:
        y_test_pred = model.predict(X_test)
    if count == place:
        X_test = tk.texts_to_matrix(df_dataK['text'], mode='tfidf')
        y_test_pred = model.predict(X_test)
    print (" Test accuracy = {}" 
        .format(round(accuracy_score(y_test, y_test_pred),3)))

    count += 1

LogisticRegression()
 Test accuracy = 0.802
SGDClassifier()
 Test accuracy = 0.87
RandomForestClassifier()
 Test accuracy = 0.843
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
 Test accuracy = 0.849
 Test accuracy = 0.126
